## Notebook for plotting profiles from pre-trained models and using DeepSHAP and TF-MoDISco to compute and visualize importance scores

In [1]:
# modify this for your own machine
%env CUDA_VISIBLE_DEVICES = 0
%config Completer.use_jedi = False

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import sys
# append paths pointing to data directory on your machine
sys.path.append('/home/katie/bp_repo/research/')
sys.path.append('/home/katie/bp_repo/multitask_profile_model_SPI_GATA/')

import os
import copy

# experimental
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
# without this: time for tf-modisco on CTCF chip-seq is: 24 mins 43 seconds
# with this: time for tf-modisco on CTCF chip-seq is: 20 mins 33 seconds and  1 hr 40 mins for full dataset
# with this: time for tf-modisco on FOSL2 chip-seq is: 5 mins 45 seconds and 30 mins for full dataset

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyfaidx
import pyBigWig
import tqdm
import glob

import profile_models
from profile_models import place_tensor
import profile_performance

import all_functions 
from all_functions import *

# modify this for your own directory
os.chdir('/home/katie/bp_repo/research/')

device = torch.device("cuda") if torch.cuda.is_available() \
        else torch.device("cpu")

# Loading in pre-trained models

### ChIP-seq

In [ ]:
CTCF_10_epochs_chip_seq_AUG1 = ModelLoader(True, 1, '/home/katie/bp_repo/research/trained_models/CTCF_10_epochs_chip_seq_AUG1.model').load_model()

In [ ]:
FOSL2_10_epochs_chip_seq_AUG1 = ModelLoader(True, 1, '/home/katie/bp_repo/research/trained_models/FOSL2_10_epochs_chip_seq_AUG1.model').load_model()

### CUT&RUN

In [ ]:
CTCF_10_epochs_cutnrun_single_task_maxfl120_AUG5 = \
ModelLoader(False, 1, '/home/katie/bp_repo/research/trained_models/CTCF_10_epochs_cutnrun_single_task_maxfl120_AUG5.model').load_model()

In [ ]:
CTCF_10_epochs_cutnrun_single_task_minfl150_AUG5 = \
ModelLoader(False, 1, '/home/katie/bp_repo/research/trained_models/CTCF_10_epochs_cutnrun_single_task_minfl150_AUG5.model').load_model()

In [ ]:
CTCF_10_epochs_cutnrun_multi_task_AUG5 = \
ModelLoader(False, 2, '/home/katie/bp_repo/research/trained_models/CTCF_10_epochs_cutnrun_multi_task_AUG5.model').load_model()

In [ ]:
CTCF_10_epochs_cutnrun_single_task_AUG2 = \
ModelLoader(False, path='/home/katie/bp_repo/research/trained_models/CTCF_10_epochs_cutnrun_single_task_AUG2.model').load_model()

In [ ]:
FOSL2_10_epochs_cutnrun_single_task_AUG1 = \
ModelLoader(False, path='/home/katie/bp_repo/research/trained_models/FOSL2_10_epochs_cutnrun_single_task_AUG1.model').load_model()

In [ ]:
FOSL2_10_epochs_cutnrun_multi_task_AUG8 = \
ModelLoader(False, 2, '/home/katie/bp_repo/research/trained_models/FOSL2_10_epochs_cutnrun_multi_task_AUG8.model').load_model()

# Dataloading

### ChIP-seq

In [ ]:
tasks_path = '/home/katie/bp_repo/research/data/chip-seq/'
set_tasks_path(tasks_path)

In [ ]:
ctcf_chipseq_full =  DataLoader(['CTCF'], 'chip-seq', True, tasks_path, ['full'], jitter=False).make_loaders()['full']

In [ ]:
fosl2_chipseq_full =  DataLoader(['FOSL2'], 'chip-seq', True, tasks_path, ['full'], jitter=False).make_loaders()['full']

### CUT&RUN 

In [ ]:
tasks_path = '/home/katie/bp_repo/research/data/cutnrun/'
set_tasks_path(tasks_path)

In [ ]:
ctcf_cutnrun_maxfl120 = DataLoader(['CTCF_120'], 'cutnrun', False, tasks_path, ['full'], jitter=False).make_loaders()['full']

In [ ]:
ctcf_cutnrun_minfl150 = DataLoader(['CTCF_150'], 'cutnrun', False, tasks_path, ['full'], jitter=False).make_loaders()['full']

In [ ]:
ctcf_cutnrun_multitask = DataLoader(['CTCF_120','CTCF_150'], 'cutnrun', False, tasks_path, ['full'], jitter=False).make_loaders()['full']

In [ ]:
ctcf_cutnrun_full = DataLoader(['CTCF'],'cutnrun',False,get_tasks_path(), ['full'], jitter=False).make_loaders()['full']

In [ ]:
fosl_cutnrun_full = DataLoader(['FOSL2'],'cutnrun',False,get_tasks_path(), ['full'], jitter=False).make_loaders()['full']

In [ ]:
fosl_cutnrun_maxfl120 = DataLoader(['FOSL2_120'], 'cutnrun', False, tasks_path, ['full'], jitter=False).make_loaders()['full']

In [ ]:
fosl_cutnrun_minfl150 = DataLoader(['FOSL2_150'], 'cutnrun', False, tasks_path, ['full'], jitter=False).make_loaders()['full']

In [ ]:
fosl_cutnrun_multitask = DataLoader(['FOSL2_120','FOSL2_150'], 'cutnrun', False, tasks_path, ['full'], jitter=False).make_loaders()['full']

# Performance evaluation

In [ ]:
import plotting_helper
from plotting_helper import *

In [ ]:
preds = get_predictions(ctcf_chipseq_full, CTCF_10_epochs_chip_seq_AUG1)
save_preds(ctcf_chipseq_full, CTCF_10_epochs_chip_seq_AUG1, \
           '/home/katie/bp_repo/research/preds/ctcf_chipseq_full_10_epochs_sep5')

In [ ]:
model_task_stats, stats = DataLoader(['CTCF_120','CTCF_150'], 'cutnrun', False, tasks_path, ['test'], jitter=False).make_statuses()
stats = stats['test']

In [ ]:
model_task_statuses, statuses = DataLoader(['CTCF_120','CTCF_150'], 'cutnrun', False, tasks_path, ['full'], jitter=False).make_statuses()

In [ ]:
# Model is single-task ProfilePredictorWithoutControls trained for 10 epochs on FOSL2 CUT&RUN
calculate_metrics(ctcf_cutnrun_maxfl120, CTCF_10_epochs_cutnrun_multi_task_AUG5, statuses, model_task_statuses)

In [ ]:
plot_kwargs = {'tasks':['CTCF'], 
               'loaders':[ctcf_chipseq_full], 
               'models':[CTCF_10_epochs_chip_seq_AUG1], 
               'normalize':False}
plotter = ProfilePlotter(**plot_kwargs)

rng = np.random.RandomState(20210405)
loader_len = 2 * len(ctcf_chipseq_full.dataset.coords)
example_inds = rng.choice(loader_len, size=min(5, loader_len), replace=False)
for index in example_inds:
    plotter.plot_task(task='CTCF',index=index,separate_figs=True,titles=True)#,
                                    #save_path=f'/home/katie/bp_repo/wiki-images/2021-07-08-GATA2-{index}-sep.png')

In [ ]:
plot_kwargs = {'tasks':['CTCF_150'], 
               'loaders':[ctcf_cutnrun_minfl150_test], 
               'models':[CTCF_10_epochs_cutnrun_single_task_minfl150_AUG5], 
               'normalize':True}
plotter = ProfilePlotter(**plot_kwargs)

rng = np.random.RandomState(20210405)
loader_len = 2 * len(ctcf_cutnrun_minfl150_test.dataset.coords)
example_inds = rng.choice(loader_len, size=min(5, loader_len), replace=False)
for index in example_inds:
    plotter.plot_task(task='CTCF_150',index=index,separate_figs=True,titles=True)#,
                                    #save_path=f'/home/katie/bp_repo/wiki-images/2021-07-08-GATA2-{index}-sep.png')

In [ ]:
plot_kwargs = {'tasks':['CTCF_multi'], 
               'loaders':[ctcf_cutnrun], 
               'models':[CTCF_10_epochs_cutnrun_single_task_maxfl120_AUG5], 
               'normalize':True}
plotter = ProfilePlotter(**plot_kwargs)

rng = np.random.RandomState(20210405)
loader_len = 2 * len(ctcf_cutnrun_maxfl120_test.dataset.coords)
example_inds = rng.choice(loader_len, size=min(5, loader_len), replace=False)
for index in example_inds:
    plotter.plot_task(task='CTCF_120',index=index,separate_figs=True,titles=True)#,
                                    #save_path=f'/home/katie/bp_repo/wiki-images/2021-07-08-GATA2-{index}-sep.png')

# SHAP and TF-MoDISco

In [ ]:
sys.path.append('/home/katie/bp_repo/')
sys.path.append('/home/katie/bp_repo/shap_modisco_scripts/')
sys.path.append('/home/katie/bp_repo/shap/')

## IF YOU NEED TO INSTALL ALEX'S VERSION OF SHAP
#!git clone https://github.com/amtseng/shap /home/katie/bp_repo/shap
#!pip install /home/katie/bp_repo/shap

import viz_sequence
import compute_shap
from importlib import reload

import modisco
import h5py

import shap_modisco_helper
from shap_modisco_helper import *

import viz_tf_modisco_results
from viz_tf_modisco_results import *

from datetime import datetime

In [ ]:
start = datetime.now()
make_shap_scores('/home/katie/bp_repo/research/trained_models/CTCF_10_epochs_chip_seq_AUG1.model',
                'profile', ctcf_chipseq_full, 2114, 1, '/home/katie/bp_repo/modisco_results/CTCF/shap_scores_chipseq_full_sep5',
                '/home/katie/bp_repo/multitask_profile_model_SPI_GATA/data/genomes/hg38.fasta',
                '/home/katie/bp_repo/research/data/hg38.chrom.sizes', controls='matched')
end = datetime.now()
end-start

In [ ]:
# better to run this with bigger GPU
start = datetime.now()
run_tf_modisco('/home/katie/bp_repo/modisco_results/FOSL2/shap_scores_cutnrun_multitask_minfl150',
    '/home/katie/bp_repo/modisco_results/FOSL2/tfmodisco_results_cutnrun_multitask_minfl150',
    '/home/katie/bp_repo/modisco_results/FOSL2/seqlets_cutnrun_multitask_minfl150', 400)
end = datetime.now() 
end - start

In [ ]:
# better to run this with bigger GPU - CURRENT
start = datetime.now()
run_tf_modisco('/home/katie/bp_repo/modisco_results/CTCF/shap_scores_chipseq_full_sep25',
    '/home/katie/bp_repo/modisco_results/CTCF/tfmodisco_results_chipseq_full_sep25',
    '/home/katie/bp_repo/modisco_results/CTCF/seqlets_chipseq_full_sep25', 400)
end = datetime.now() 
end - start

In [ ]:
# NEXT UP
start = datetime.now()
run_tf_modisco('/home/katie/bp_repo/modisco_results/FOSL2/shap_scores_chipseq_full_sep25',
    '/home/katie/bp_repo/modisco_results/FOSL2/tfmodisco_results_chipseq_full_sep25',
    '/home/katie/bp_repo/modisco_results/FOSL2/seqlets_chipseq_full_sep25', 400)
end = datetime.now() 
end - start

### TF-MoDISco result visualization

In [ ]:
# Multi-task model trained on CTCF CUT&RUN - tasks are maxfl120 and minfl150
# SHAP scores/tf-modisco computed only for maxfl120 task
# expected motif: CCACCAGGGGG (approximately)
viz_motif('/home/katie/bp_repo/modisco_results/CTCF/tfmodisco_results_cutnrun_multitask_maxfl120')

In [ ]:
# Multi-task model trained on CTCF CUT&RUN - tasks are maxfl120 and minfl150
# SHAP scores/tf-modisco computed only for minfl150 task
# expected motif: CCACCAGGGGG (approximately)
viz_motif('/home/katie/bp_repo/modisco_results/CTCF/tfmodisco_results_cutnrun_multitask_minfl150')

In [ ]:
# Multi-task model trained on FOSL2 CUT&RUN - tasks are maxfl120 and minfl150
# SHAP scores/tf-modisco computed only for maxfl120 task
# expected motif: TGACTCA / TGAGTCA / TGACGTCA
viz_motif('/home/katie/bp_repo/modisco_results/FOSL2/tfmodisco_results_cutnrun_multitask_maxfl120')

In [ ]:
# Multi-task model trained on FOSL2 CUT&RUN - tasks are maxfl120 and minfl150
# SHAP scores/tf-modisco computed only for minfl150 task
# expected motif: TGACTCA / TGAGTCA / TGACGTCA
viz_motif('/home/katie/bp_repo/modisco_results/FOSL2/tfmodisco_results_cutnrun_multitask_minfl150')

In [ ]:
# Multi-task model trained on CTCF CUT&RUN
# SHAP scores/tf-modisco computed only for maxfl120 task
# expected motif: CCACCAGGGGG (approximately)
viz_motif('/home/katie/bp_repo/modisco_results/CTCF/tfmodisco_results_cutnrun_multitask_maxfl120')

In [ ]:
# Multi-task model trained on CTCF CUT&RUN
# SHAP scores/tf-modisco computed only for minfl150 task
# expected motif: CCACCAGGGGG (approximately)
viz_motif('/home/katie/bp_repo/modisco_results/CTCF/tfmodisco_results_cutnrun_multitask_minfl150')

In [ ]:
# Single-task model trained on CTCF CUT&RUN
# SHAP scores/tf-modisco computed only for CTCF task
# expected motif: CCACCAGGGGG (approximately)
viz_motif('/home/katie/bp_repo/modisco_results/CTCF/tfmodisco_results_ctcf_full_cutnrun')

In [ ]:
# Single-task model trained on FOSL2 CUT&RUN
# SHAP scores/tf-modisco computed only for FOSL2 task
# expected motif: TGACTCA / TGAGTCA / TGACGTCA
viz_motif('/home/katie/bp_repo/modisco_results/FOSL2/tfmodisco_results_fosl_full_cutnrun')

In [ ]:
# Single-task model trained on CTCF ChIP-seq
# SHAP scores/tf-modisco computed only for the CTCF task
# expected motif: CCACCAGGGGG (approximately)
viz_motif(tfm_results_path_ctcf_chip_seq)

In [ ]:
# Single-task model trained on FOSL2 ChIP-seq
# SHAP scores/tf-modisco computed only for the FOSL2 task
# expected motif: TGACTCA / TGAGTCA / TGACGTCA
viz_motif(tfm_results_path_fosl_chip_seq)

## histogram plot of cut&run fragment lengths

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
import sklearn

In [ ]:
di = pickle.load(open('frag_lengths_histogram.dict','rb'))

with open('data/cutnrun/CTCF_frag_lengths.txt','r') as ff:
    c = ff.readlines()
with open('data/cutnrun/FOSL2_frag_lengths.txt','r') as ff:
    d = ff.readlines()
    
c = [int(l.strip()) for l in c] # len 39478628
d = [int(l.strip()) for l in d] # len 22967252

histogram_dict = {}
titles = ['CTCF fragment lengths', 'FOSL2 fragment lengths']

histogram = plt.hist(np.array(c), bins=20)
plt.xlabel('Fragment length')
plt.ylabel('Number of fragments')
plt.title(titles[0])
histogram_dict[titles[0]] = plt.gcf

#pickle.dump(histogram_dict, open('frag_lengths_histogram.dict','wb'))

In [ ]:
with open('/home/katie/bp_repo/research/data/cutnrun/temp/fragment_lengths/CTCF_frag_lengths.txt','r') as ff:
    ctcf = ff.readlines()
with open('/home/katie/bp_repo/research/data/cutnrun/temp/fragment_lengths/FOSL2_frag_lengths.txt','r') as ff:
    fosl = ff.readlines()
    
ctcf = [int(l.strip()) for l in ctcf] # len 39478628
fosl = [int(l.strip()) for l in fosl] # len 22967252

ctcf = np.array(ctcf).reshape((-1,1))
fosl = np.array(ctcf).reshape((-1,1))

In [ ]:
len(ctcf)

In [ ]:
ctcf = np.array(ctcf).reshape((-1,1))
fosl = np.array(ctcf).reshape((-1,1))

In [ ]:
ctcf_cluster = sklearn.cluster.MiniBatchKMeans(n_clusters=3)

In [ ]:
start = datetime.now()
ctcf_cluster.fit(ctcf)
print(datetime.now() - start)

In [ ]:
ctcf_preds = ctcf_cluster.predict(ctcf)

In [ ]:
sum(ctcf[:,0] > 400)

In [ ]:
pd.value_counts(ctcf_preds)

In [ ]:
ctcf_cluster.cluster_centers_

In [ ]:
ctcf_preds_prob = ctcf_cluster.

In [ ]:
pd.Series(ctcf[:,0]).describe()

In [ ]:
di['CTCF fragment lengths']